# Finding records

Projects quickly accumulate thousands of recorded tasks, and finding particular results can be like finding a needle in a haystack. In theory all of the execution information is recorded in a Sumatra *data store* (in particular the execution time and duration, task and parameters, and the location of output files); the examples below show different methods to query that database and reload results from within a Jupyter notebook.

> By design, only *RecordedTasks* are saved to the Sumatra database. *MemoizedTasks* are not.

**TODO**: Create a demo project within the *smttask* repo and use that instead of *IndEEG*.

**NOTE**: The visualization tools have improved since this document was written. Until we have proper auto-built API documentation, for the most up to date version, please peruse the source code of [smttask.view.recordstoreviewer](../smttask/view/recordstoreviewer.py).

In [1]:
import IndEEG
IndEEG.setup('theano', view_only=True)
import sinn

In [2]:
import smttask
import smttask.utils
from tqdm.auto import tqdm
from IndEEG.parameters import ParameterSet  # Customized version of parameters.ParameterSet
from mackelab_toolbox.utils import print_api
from mackelab_toolbox.parameters import dfdiff, ParameterComparison

*smttask* provides the `RecordStoreView` class for interfacing with the record store. It can be called without arguments if the current directory is within the tracked project.

In [3]:
rsview = smttask.RecordStoreView().filter.tags("finished")

> Smttask follows the behaviour of Sumatra and automatically tags records during execution, to track their status. These status tags are:
>   + **\_\_initialized\_\_**  — Task terminated before starting to run
>   + **\_\_running\_\_**  – Task is still running
>   + **\_\_crashed\_\_**   – Task terminated prematurely with an error
>   + **\_\_killed\_\_**   – Task was killed
>   + **\_\_finished\_\_** – Task completed successfully.
>
> An initial filter for **\_\_finished\_\_** tags is computationally very cheap, and avoids iterating over incomplete runs.
> The filtering mechanism is explained [below](#Filtering).

`RecordStoreView` wraps an iterable over the records, which may or may not be consumable. Without any additional filtering, this iterable is over the entire record store.
Iterating over records can be slow; calling `.list` on the record store view will make it cache the records internally as a list, much accelerating further iterations.

In [4]:
rsview.list;

## Record list summary

`RecordStoreView` provides the property `summary` (of type `RecordStoreViewSummary`), which in a notebook displays as a Pandas *Dataframe* summarising the records. This is a good way to get an initial overview of a data store, or to view the result after the list has been filtered (see [Filtering](#Filtering)).

The output can be adjust with the following methods:

- `merged` (property): Combine records with similar labels (by default, the same timestamp; type `RecordStoreViewSummary?` for instructions for how to change the merge pattern). The number of merged records is displayed in each row.
- `unmerged` (property): Inverts the `merged` operation.
- `head(nrows)`: Restrict the summary to the first `nrows`.
- `tail(nrows)`: Restrict the summary to the last `nrows`.
- `dataframe(...)`: Return the *Dataframe* used for display. Arguments are provided to adjust the content:
  + *fields*: Which fields to include as columns in the dataframe. Default: *reason*, *outcome*, *tags*, *main_file* *duration*.
  + *parameters*: Which parameters to include; specified as tuples of string; snested parameters can be specified with dots.
  + *max_chars*: Truncate columns to this number of characters.
  + *max_lines*: Keep only this number of lines from a field, even if more lines would fit within the character limit.

In [5]:
rsview.summary.merged.tail(15)

,# records,reason,outcome,tags,main_file,avg. duration
20201006-142712,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,3h 52m 39s
20201006-125319,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,1h 44m 02s
20201006-125318,2,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,2h 03m 04s
20201006-125317,11,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,2h 38m 18s
20201005-103123,2,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,2h 36m 13s
20201005-103122,11,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,3h 57m 15s
20201004-202839,2,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,2h 02m 09s
20201004-202838,9,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,3h 16m 14s
20200924-235015,1,Test fix to rightmost batch,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,6h 35m 55s
20200916-162115,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,3h 55m 18s


## Basic record selection

- `.get()`: Return the record(s) matching a specific label(s).
- `.earliest`: Return the earliest record.
- `.latest`: Return the latest record. A simple way to obtain an individual record.
- `.list`: Make the RecordStoreView non-consuming (convert its iterable to a list). This is done in-place. Avoids querying the record store for subsequent iterations.
- Standard “smart” indexing (i.e `rsview[key]`): Uses some heuristics to determine what to index:
  + By label, if *key* is a str. Equivalent to `.get(key)`.
  + The cached `.list`, if it is available and *key* is an int. Equivalent to `.list[key]`.
  + The underlying iterable, otherwise. (No public equivalent, but can be achieved with `._iterable[key]`.) \
  This is provided as a convenience during exploration.

In [6]:
rsview.latest

Record #20201119-112247_f334b8

In [7]:
rsview.get('20201118-212254_c2c6ab')

Record #20201118-212254_c2c6ab

In [8]:
rsview.get(['20201118-212254_c2c6ab', '20201119-095543_8b5fc1'])

[Record #20201118-212254_c2c6ab, Record #20201119-095543_8b5fc1]

In [9]:
rsview['20201118-212254_c2c6ab']

Record #20201118-212254_c2c6ab

In [10]:
rsview[0]

Record #20201119-112247_f334b8

## Sumatra.RecordStore interface

`RecordStoreView` also reproduces the part of the interface provided by Sumatra's `RecordStore` which makes senses for a read-only view.

| *smttask.RecordStoreView* | *sumatra.recordstore.RecordStore* |  Description |
|:---|:---|:---|
| `.aslist()`    | `.list(...)`  | Return the records as a list. |
| `.labels()`    | `.labels(...)` | Return the list of record labels (RecordStoreView caches the value). |
| `.most_recent()` | `.most_recent(...)` | Return the *label* of the most recent record. Equivalent to `.latest.label`. |
| `.export(indent=2)` | `.export(...)` |  Return a string with a JSON representation of the project record store. |
| `.export_records(records, indent=2)` | `.export_records(...)` | Return a string with a JSON representation of the given records |


### Exception to read-only interface

`RecordStoreView` also adds `add_tag` and `remove_tag` methods, which modifying the underlying recorstore by respectively adding and removing tags to every record in the view. Combined with [filtering](#Filtering), this is an efficient way to mark particular records for later access, especially because tag filters are by far the [fastest](#Pre-vs-post-filters). For example, one can tag all records required for a particular figure:
```python
rsview.filter.[date/version/parameter conditions].add_tag('figure1')
```
Retrieving those records can then be done in milliseconds, even with a store containing thousands of records:
```python
records = rsview.filter.tag('figure1').filter.[panel 1 condition]
```

## Filtering

The primary mechanism for pairing down the number of records is the *filter*. Filters can be chained, so for example to select all records between September 12th (inclusive) and 16th (exclusive) 2020:

In [11]:
records = (rsview.filter.after(20200912)
                 .filter.before(20200916)
          ).list
records.summary

,# records,reason,outcome,tags,main_file,avg. duration
20200915-164710_90be,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,4h 20m 17s
20200915-164710_4e31,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,2h 34m 51s
20200915-154121_d464,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,4h 53m 32s
20200915-154121_a0fa,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,2h 45m 02s
20200912-132121_d712,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,0h 21m 17s
20200912-132121_c390,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,1h 16m 56s
20200912-132121_773d,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,1h 23m 30s


> Filters return generators (to make chaining cheap), which is why we use the `.list` on the result to avoid the RecordStoreView being consumed the first time we use it.

### Builtin filters

The builtin filters are listed below; they are all accessed as attributes, as `filter.<filter name>`.

In [12]:
for fltr in rsview.filter.registered_filters.values():
    print_api(fltr)

generic_filter(fn: Callable)
    The default filter: keep records for which `fn` returns True.
    Equivalent to Python's `filter`.

before(date, *args)
    Keep only records which occured before the given date. Date is exclusive.
    Can provide date either as a single tuple, or multiple arguments as for
    `datetime.datetime()`; a `datetime` instance is also accepted.

    As a convenience, tuple values may be concatenated and replaced by a
    single integer of 4 to 8 digits; if it has less than 8 digits, it is
    extended to the earliest date (so 2018 -> 20180101).

after(date, *args)
    Keep only records which occurred after the given date. Date is inclusive.
    Can provide date either as a single tuple, or multiple arguments as for
    `datetime.datetime()`; a `datetime` instance is also accepted.

    As a convenience, tuple values may be concatenated and replaced by a
    single integer of 4 to 8 digits; if it has less than 8 digits, it is
    extended to the earliest date 

#### Selecting based on parameters

The filters `params` and `match` are specialized by *smttask* to recognize *task descriptions*: when specifying hierarchical parameters which include task descriptions, the *input* level may be omitted. For example, the following matches the key `'optimizer.model.params.μtilde'`, even though for some records the full key would be `'inputs.optimizer.inputs.model.inputs.params.μtilde'`. This is not only shorter, but at least in some cases will match records parameterized both with values and upstream tasks.

(Internally, the `params` and `match` filter use the `get_task_param` function [detailed below](#Retrieving-parameter-values).)

In [13]:
rsview.filter.params(
    eq={'optimizer.model.params.μtilde': [-3.4729471730413772, -0.13678844546243388]}
).summary

,# records,reason,outcome,tags,main_file,avg. duration
20201119-090726_a8a2b0,1,Test fit OUInput,,_running_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,0h 00m 00s
20201114-221233_dcfd10,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,11h 21m 51s
20201110-213127_351be7,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,4h 07m 31s
20201109-183035_bfd996,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,4h 10m 25s


In [14]:
# Full key of the first record includes three task descs:
ParameterSet(rsview['20201119-090726_a8a2b0'].parameters)[
    'inputs.optimizer.inputs.model.inputs.params.μtilde']

[-3.4729471730413772, -0.13678844546243388]

### Pre vs post filters

Most builtin filters, and all custom filters, are *post*-filters: an iterable over the record store is first created, then iterated over and the filter applied to each record. This is flexible but each query to the record store carries substantial overhead.

Fore record stores built on top of database interfaces, specifically the Django record store, many filter can in theory be integrated into the SQL query which constructs the iterator. Since in this case the filter is applied before the iterator, we call it a *pre* filter, and it can be much faster. However, support for pre-filters must be provided on a per-record store and per-filter basis; at present, only a pre-filter for `tags` is provided, since that is the one also provided by Sumatra.

Available pre-filters are applied automatically, *as long as they appear before any post-filter*. As an example, compare the execution time of the two following queries, which differ only in the order of their filters.

In [15]:
rsview2 = smttask.RecordStoreView()

In [16]:
%time rsview2.filter.tags('killed').filter.label('202011').list;

CPU times: user 133 ms, sys: 4.01 ms, total: 137 ms
Wall time: 135 ms


In [17]:
%time rsview2.filter.label('202011').filter.tags('killed').list;

CPU times: user 4.55 s, sys: 127 ms, total: 4.67 s
Wall time: 4.67 s


> **tl;dr**: Some filters have *pre* versions. Apply those first.

### Custom filters

Rather than using one of the builtin filters listed above, one may instead pass an arbitrary function to the `.filter` attribute. This function should take one argument (the record) and return a bool; records for which it returns `True` are kept.

For example, to kept only records whose duration was greater than 10 hours, one could do (duration is recorded in seconds):

> `rsview.filter(...)` and `rsview.filter.generic_filter(...)` are semantically equivalent.

In [18]:
records = rsview.filter.output().filter(lambda rec: rec.duration > 10*60*60).list
records.summary

,# records,reason,outcome,tags,main_file,avg. duration
20201118-145225_517742,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,19h 03m 05s
20201118-141502_35e1c8,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,15h 50m 42s
20201118-140625_67a79e,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,17h 22m 25s
20201118-124009_bf49cc,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,18h 20m 00s
20201118-121208_5caff8,1,Test fit OUInput,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,20h 55m 06s
...,...,...,...,...,...,...
20201028-211334_058f,1,Diagnose – param-only fit,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,10h 58m 19s
20201014-090309_e5d8,1,Test: no dynamics on rightmost latents batch,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,10h 18m 57s
20201012-014504_1edb,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,10h 29m 44s
20201011-121232_b536,1,Hyperparameter exploration,,_finished_,…cherche/Individualized_EEG/IndEEG/IndEEG/tasks.py,11h 06m 31s


## Comparing records

### Binary comparisons

Let's say you have two records, *record1* and *record2*, which were produced using almost identical parameterisations but gave different results.

In [19]:
record1 = rsview.get('20201119-075320_7b84e7')
record2 = rsview.get('20201119-042150_e177c4')

If you don't remember exactly how each was run (or misremember), how do you determine what might explain the difference between the results ? Simply comparing the two parameter sets by eye is virtually impossible if there are more than a handful of parameters.

In [20]:
params1 = ParameterSet(record1.parameters)
params2 = ParameterSet(record2.parameters)

In [21]:
# Commented out for brevity
#print(params1.pretty())

The *mackelab_toolbox* provides the function `dfdiff` for comparing two parameter sets. It works with hierarchical parameter sets, and keeps only those entries which differ. The result is returned as a Pandas Dataframe, so it displays nicely and can be further indexed.

In [22]:
dfdiff(params1, params2)

pset 1  \
inputs digest        –                 –      –                                7b84e7ecbb__nsteps_25000   
       recorders     –                 –      –       [{'name': 'log L', 'keys': None, 'interval': 1...   
       hashed_digest –                 –      –                                              7b84e7ecbb   
       optimizer     latent_cache_path –      –                                .cache/latents_process-2   
                     model             params μtilde         [-0.2644939661170402, -0.6205617416379619]   
                                              τtilde            [2.5375870766829345, 2.209510172754845]   
                                              σtilde           [2.297311061527454, 0.11218153077399037]   
                                              Wtilde  [[1.0581511200534628, -0.6645030719061765], [-...   
reason –             –                 –      –       Test fit OUInput\n\nObserved latents\n\n- OU +...   

                                                                                                 pset 2  
inputs digest        –                 –      –                                e177c4e9ca__nsteps_25000  
       recorders     –                 –      –       [{'name': 'log L', 'keys': None, 'interval': 1...  
       hashed_digest –                 –      –                                              e177c4e9ca  
       optimizer     latent_cache_path –      –                                .cache/latents_process-4  
                     model             params μtilde         [-2.261403657553197, -0.11352527517905774]  
                                              τtilde          [0.23277087700432836, 1.4732451278346765]  
                                              σtilde           [2.276408794306698, 0.08502108251737077]  
                                              Wtilde  [[-0.11604415987541336, -1.192318168175603], [...  
reason –             –                 –      –       Test fit OUInput\n\nObserved latents\n\n- OU +...

In [23]:
dfdiff(params1, params2).sort_index().loc[('inputs','optimizer','model')]

pset 1  \
params Wtilde  [[1.0581511200534628, -0.6645030719061765], [-...   
       μtilde         [-0.2644939661170402, -0.6205617416379619]   
       σtilde           [2.297311061527454, 0.11218153077399037]   
       τtilde            [2.5375870766829345, 2.209510172754845]   

                                                          pset 2  
params Wtilde  [[-0.11604415987541336, -1.192318168175603], [...  
       μtilde         [-2.261403657553197, -0.11352527517905774]  
       σtilde           [2.276408794306698, 0.08502108251737077]  
       τtilde          [0.23277087700432836, 1.4732451278346765]

### Comparing multiple records

The *mackelab_toolbox* also provides the `ParameterComparison` object, which is not limited to binary comparisons and works directly on either records or parameter sets. Below we compare all the records executed on the 10th of November 2020:

> `ParameterComparison` is essentially doing an outer product of all key/value pairs in all parameter sets, and storing any difference. In the worst case, the memory requirements can therefore be exponential in the number of records compared.

In [24]:
cmp = ParameterComparison(rsview.filter.on(20201110))

To display the results of the comparison, use the `.dataframe()` method. By default, differences in hierarchical parameters are folded (indicated by `<+>`) to keep things legible even with large hierarchichal parameter sets.

In [25]:
cmp.dataframe()

,inputs,reason
20201110-213237_87825b,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-213215_acd030,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-213133_4ca955,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-213127_351be7,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-213116_cce369,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
...,...,...
20201110-172330_3a1378,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-172330_0dc62d,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-172330_de06e3,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...
20201110-172330_8ad9ee,<+>,Test fit OUInput\n\nObserved latents\n\n- OU +...


We use the `depth` keyword argument to drill down into the the comparison.

Often when doing this we want to hide certain columns; for example, since the *reason* column is free-form text, it may not useful in determining what caused two computations to differ. Since this is a standard *Dataframe*, we can hide columns with `.drop(columns=...)`.

In [26]:
cmp.dataframe(depth=2).drop(columns=['reason'])

,inputs.hashed_digest,inputs.recorders,inputs.optimizer,inputs.digest
20201110-213237_87825b,87825b4f90,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,87825b4f90__nsteps_25000
20201110-213215_acd030,acd0303f20,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,acd0303f20__nsteps_25000
20201110-213133_4ca955,4ca955e136,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,4ca955e136__nsteps_25000
20201110-213127_351be7,351be75bdc,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,351be75bdc__nsteps_25000
20201110-213116_cce369,cce369d5f6,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,cce369d5f6__nsteps_25000
...,...,...,...,...
20201110-172330_3a1378,3a137881ef,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,3a137881ef__nsteps_25000
20201110-172330_0dc62d,0dc62d9f89,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,0dc62d9f89__nsteps_25000
20201110-172330_de06e3,de06e30753,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,de06e30753__nsteps_25000
20201110-172330_8ad9ee,8ad9ee4be0,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,8ad9ee4be0__nsteps_25000


> **Hint**: Don't underestimate the value of recording useful information in a task's “reason” attribute. Below are the recorded “reasons” for the first 4 entries in the same record list:

In [40]:
for rec, _ in zip(records, range(4)):
    print(f"----- {rec.label} -------")
    print("\n".join(rec.reason))
    print("")

----- 20201110-213237_87825b -------
Test fit OUInput

Observed latents

- OU + Linear projection
- Synthetic data
- Init params: (5, 33)
- 25000 passes
- params: {'λθ': 0.0001}

----- 20201110-213215_acd030 -------
Test fit OUInput

Observed latents

- OU + Linear projection
- Synthetic data
- Init params: (5, 34)
- 25000 passes
- params: {'λθ': 0.0001}

----- 20201110-213133_4ca955 -------
Test fit OUInput

Observed latents

- OU + Linear projection
- Synthetic data
- Init params: (5, 25)
- 25000 passes
- params: {'λθ': 0.0002}

----- 20201110-213127_351be7 -------
Test fit OUInput

Observed latents

- OU + Linear projection
- Synthetic data
- Init params: (5, 55)
- 25000 passes
- params: {'λθ': 0.0001}



### Limitation

Parameters for a task *T* can be specified either as values or as other tasks. In the latter case, the serialization of the task creates a *task description* with the keys *taskname*, *module* and *inputs*. The *inputs* entry can itself contain the serialization of other tasks – thus is the entire specification for *T* saved and recoverable from the record store.

However, this does make it more difficult to compare records if in some cases parameters are specified as values, and in others as task descriptions – since those can never be equal.

The function `fold_task_inputs()` (found in *smttask.utils*) can help working with these nested parameter sets created by chained tasks: it replaces task descriptions by the contents of their *inputs*. (In many cases the *taskname* is the same for all records.) This doesn't solve the problem of values differing from task descriptions, but at least the latter don't create such deeply nested hierarchies.

In [28]:
records = rsview.filter.on(20201110).list
cmp = ParameterComparison(params=[smttask.utils.fold_task_inputs(rec.parameters)
                                  for rec in records],
                          labels=[rec.label for rec in records])

In [29]:
cmp.dataframe(depth=1)

,hashed_digest,recorders,optimizer,digest
20201110-213237_87825b,87825b4f90,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,87825b4f90__nsteps_25000
20201110-213215_acd030,acd0303f20,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,acd0303f20__nsteps_25000
20201110-213133_4ca955,4ca955e136,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,4ca955e136__nsteps_25000
20201110-213127_351be7,351be75bdc,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,351be75bdc__nsteps_25000
20201110-213116_cce369,cce369d5f6,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,cce369d5f6__nsteps_25000
...,...,...,...,...
20201110-172330_3a1378,3a137881ef,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,3a137881ef__nsteps_25000
20201110-172330_0dc62d,0dc62d9f89,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,0dc62d9f89__nsteps_25000
20201110-172330_de06e3,de06e30753,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,de06e30753__nsteps_25000
20201110-172330_8ad9ee,8ad9ee4be0,"[{'name': 'log L', 'keys': None, 'interval': 1...",<+>,8ad9ee4be0__nsteps_25000


## Retrieving parameter values

Serialization establishes an equivalence between `Tasks`, *task descriptions* and `ParameterSet`s, but each has their own syntax to retrieve particular parameters. This is especially cumbersome with nested structures, where these types can arbitrarily mix. The function `get_task_param` (again from *smttask.utils*) provides a unique syntax that works with every object, and supports nested selection.

In [30]:
smttask.utils.get_task_param(records.latest, 'optimizer.model.params.μtilde')    # Record

[3.3860280584311826, -0.11294867494456073]

In [31]:
smttask.utils.get_task_param(records.latest.parameters,                          # ParameterSet
                             'optimizer.model.params.μtilde')

[3.3860280584311826, -0.11294867494456073]

In [33]:
smttask.utils.get_task_param(smttask.Task.from_desc(records.latest.parameters),  # Task
                             'optimizer.model.params.μtilde').get_value()

array([ 3.38602806, -0.11294867])